In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from scipy import ndimage
from scipy import integrate
from astropy import units as u
import pandas as pd
from astropy.convolution import interpolate_replace_nans

In [2]:
parameters=pd.read_csv('Parameters.csv')
dist=parameters['Distance']
inc=parameters['Inclination']
PA=parameters['Position angle']
freq=parameters['Frequency']
beamwidth=parameters['Beam width']
channelwidth=parameters['Channel width']
xcentre=parameters['X centre']
ycentre=parameters['Y centre']
N1=parameters['N1']
N2=parameters['N2']
R=parameters['Ratio']
filepath=parameters['File Path']
beamtable=parameters['Beam Table']
havecube=parameters['Cube']
assize=parameters['Total Size']
dims=parameters['Dims']
jupper=parameters['Jupper']

In [3]:
def total_size(distance,sizeas):
    totsize=4.84*distance*sizeas
    return totsize

In [4]:
def get_info(hdu,distance,beamtable):
    hdr=hdu[0].header
    if beamtable=='Yes':
        beamtab=hdu[1].data
        bmajas=np.median(beamtab['BMAJ'])
        bminas=np.median(beamtab['BMIN'])
        bmaj=bmajas/3600
        bmin=bminas/3600
    else:
        bmaj=hdr['BMAJ']
        bmin=hdr['BMIN']
    xcellsize=hdr['CDELT1']
    deltx_arcsec=np.abs(xcellsize)*3600
    deltx_pc=4.84*distance*deltx_arcsec
    return deltx_pc,bmaj,bmin,xcellsize

In [5]:
def get_ratio(radius,inclination):
    major_axis=radius
    if inclination>85:
        minor_axis=major_axis
    else:
        minor_axis=radius*np.cos(np.deg2rad(inclination))
    ratio=major_axis/minor_axis
    return ratio

In [6]:
def distance_ellipse(shape, center, ratio, angle):

    """
    :return:
    """

    return dist_ellipse(shape, center.x, center.y, ratio, angle.to("deg").value)

def dist_ellipse(deltx_pc, my_apertute_size,n1,n2,xc, yc, ratio, pa=0): 
    ang = np.radians(pa + 90.)
    cosang = np.cos(ang)
    sinang = np.sin(ang)
    nx = n1
    ny = n2
    x = np.arange(-xc,nx-xc)
    y = np.arange(-yc,ny-yc)

    im = np.empty([n1,n2])
    xcosang = x*cosang
    xsinang = x*sinang

    for i in range(0, ny):

        xtemp = xcosang + y[i]*sinang
        ytemp = -xsinang + y[i]*cosang
        im[i,:] = np.sqrt((xtemp*ratio)**2 + ytemp**2)
    return im*deltx_pc<my_apertute_size 

In [7]:
#def get_radius(radius,deltx_pc):
    #radius_pixel=radius/deltx_pc
    #radius_pix=round(radius_pixel)
    #return radius_pix

In [8]:
#radius_pix,xcentre,ycentre
def hydrogen_mass(cube,im,distance,bmaj,bmin,xcellsize,channel_width,R,Jup):
    #mh=1.67e-27*u.kg
    #kb=1.38e-23*u.J/u.Kelvin
    Xco=3e20#*u.s/(u.Kelvin*u.km*(u.cm)**2)
    Xco20=3e20/2e20
    Rref=1/R
    Dl=distance
    #wavelength=(3e8/frequency)*u.meter
    #x_max=xcentre+radius_pix
    #x_min=xcentre-radius_pix
    #y_max=ycentre+radius_pix
    #y_min=ycentre-radius_pix
    data=im*cube
    inv_beam=(math.pi*(bmaj/np.abs(xcellsize))*(bmin/np.abs(xcellsize)))/(4*math.log(2))
    #spectrum=(inv_beam**(-1)*(cube[:,int(x_min):int(x_max),int(y_min):int(y_max)])).sum(axis=1).sum(axis=1)
    spectrum=np.nansum((np.nansum((inv_beam**(-1)*data),axis=1)),axis=1)
    #sum_spectrum=(integrate.trapz(spectrum))*u.Jy*u.km/u.s
    sum_spectrum=(np.nansum(spectrum)*channel_width)#*u.Jy*u.km/u.s
    #mass_hydrogen=np.abs((2*mh*(wavelength**2/(2*kb))*Xco*Dl**2*R*sum_spectrum).decompose())
    #mass_hydrogen_lsolar=(mass_hydrogen/(2e30*u.kg)).decompose()
    mass_hydrogen_lsolar=7847*(1/(Jup**2))*Xco20*Rref*Dl**2*sum_spectrum
    return mass_hydrogen_lsolar

In [9]:
totsize=total_size(dist,assize)

In [10]:
mass=[]
for i in range(len(havecube)):
    app=100
    if havecube[i]=='No':
        hmass=np.NaN
        mass.append(hmass)
    else:
        hdu=fits.open(filepath[i])
        cube=hdu[0].data
        if dims[i]==4:
            cuben=np.squeeze(cube)
        else:
            cuben=cube
        if beamwidth[i]>app:
            hmass=np.NaN
            mass.append(hmass)
        else:
            info=get_info(hdu,dist[i],beamtable[i])
            deltx_pc=info[0]
            bmaj=info[1]
            bmin=info[2]
            xcellsize=info[3]
            #mask=smooth_mask(dimensions[i],cube,bmaj/np.abs(xcellsize),[1,20],rmsfac=3)
            #cubem=mask*cube

            ratio=get_ratio(app,inc[i])
    
            im=dist_ellipse(deltx_pc,app,N1[i],N2[i],xcentre[i],ycentre[i],ratio,PA[i])
    
            hmass=hydrogen_mass(cuben,im,dist[i],bmaj,bmin,xcellsize,channelwidth[i],R[i],jupper[i])
    
            mass.append(hmass)
        hdu.close()
print(np.array(mass))

[           nan 7.38611785e+07 3.30474535e+08 5.11663057e+07
            nan            nan 8.59516515e+06            nan
 4.54312038e+07 1.34185112e+07 1.13242906e+07 5.51967224e+06
            nan 6.34807837e+07            nan 7.46187865e+07
 3.22332112e+07            nan 1.25509321e+07            nan
 4.31933772e+06 1.79451729e+07 6.02304604e+07 5.24522614e+07
 8.47994952e+05            nan 4.71298093e+07            nan
 1.44911643e+07            nan 1.45815918e+08 1.48138118e+07
 2.45906316e+07 2.99255931e+07            nan]


In [11]:
python=pd.read_csv('/home/jacob/Documents/Python.csv')

In [12]:
massround=np.around(np.array(mass),decimals=0)
python['Mass 75pc']=massround
print(massround)

[      nan       nan       nan 29897345.       nan       nan  5653605.
       nan 29279439.  9005715.  6289037.  4532352.       nan 39669087.
       nan 48409849. 21488192.       nan       nan       nan  2375116.
 11346039. 37602174. 33457018.   669577.       nan 25630156.       nan
 10512424.       nan 87537686.  9072066. 14911300.       nan       nan]


In [13]:
python.to_csv('Python.csv',index=False)

In [10]:
ngc=fits.open('/home/jacob/Documents/WISDOMdata/Data_cubes/NGC0404_cube.fits')
ngcdata=ngc[0].data
ngchdr=ngc[0].header

In [16]:
infongc=get_info(ngc,dist[4],beamtable[4])
deltx_pcngc=infongc[0]
bmajngc=infongc[1]
bminngc=infongc[2]
xcellsizengc=infongc[3]

In [ ]:
rationgc=get_ratio(100,inc[4])
    
imngc=dist_ellipse(deltx_pcngc,100,N1[4],N2[4],xcentre[4],ycentre[4],rationgc,PA[4])
    
hmassngc=hydrogen_mass(ngcdata,imngc,dist[4],bmajngc,bminngc,xcellsizengc,channelwidth[4],R[4],jupper[4])

In [3]:
tst=fits.open('/home/jacob/Documents/WISDOMdata/Data_cubes/NGC4826_2kms_cube.fits')

In [6]:
tsth=tst[0].header
tstd=tst[0].data

In [7]:
tstd.shape

(210, 400, 400)